#### 在Azure VM環境爬蟲台灣股市資料，匯入SSMS SQL Server

In [1]:
# step 1. 利用pandas抓取csv數據

import requests 
import pandas as pd

datestr = '20210715'
r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')
len(r.text)

2566824

In [2]:
# step 2. 根據數據格式進行切割

tickers = [] #宣告List
for i in r.text.split('\r\n'):
    if (len(i.split('",')) == 17 and i[0] != '='):
        _ticker = i.split('",') #使用",區隔每一個價格
        _ticker = [t.replace('"','') for t in _ticker] #將左邊的" 取代成空字元
        _ticker = [t.replace(',','') for t in _ticker] #將數字千分為 取代為空字元
        tickers.append(_ticker)
tickers[:100]

[['證券代號',
  '證券名稱',
  '成交股數',
  '成交筆數',
  '成交金額',
  '開盤價',
  '最高價',
  '最低價',
  '收盤價',
  '漲跌(+/-)',
  '漲跌價差',
  '最後揭示買價',
  '最後揭示買量',
  '最後揭示賣價',
  '最後揭示賣量',
  '本益比',
  ''],
 ['1101',
  '台泥',
  '11938157',
  '5094',
  '615005211',
  '51.50',
  '51.80',
  '51.30',
  '51.60',
  '+',
  '0.10',
  '51.50',
  '325',
  '51.60',
  '114',
  '12.34',
  ''],
 ['1101B',
  '台泥乙特',
  '0',
  '0',
  '0',
  '--',
  '--',
  '--',
  '--',
  ' ',
  '0.00',
  '53.40',
  '2',
  '53.90',
  '15',
  '0.00',
  ''],
 ['1102',
  '亞泥',
  '8267759',
  '3635',
  '430280018',
  '52.50',
  '52.60',
  '51.80',
  '51.90',
  '-',
  '0.60',
  '51.90',
  '146',
  '52.00',
  '2027',
  '10.16',
  ''],
 ['1103',
  '嘉泥',
  '635035',
  '446',
  '14180092',
  '22.30',
  '22.45',
  '22.25',
  '22.35',
  '+',
  '0.10',
  '22.35',
  '4',
  '22.40',
  '11',
  '7.79',
  ''],
 ['1104',
  '環泥',
  '661056',
  '421',
  '15094002',
  '23.05',
  '23.05',
  '22.75',
  '23.00',
  '+',
  '0.05',
  '22.95',
  '110',
  '23.00',
  '15',
  '10.50'

In [3]:
# step 3. 轉換成資料集格式稱之為pandas.DataFrame

# Excel Sheet(single user) = SQL Server database table (multiple users + transaction)
# DataFrame.rename(columns = {'證券代號':'Id', '證券名稱':'Name'})
df = pd.DataFrame(tickers)
df.columns = df.loc[0, :]
df.drop(0, axis = 0 , inplace = True)
df[:3]

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,
1,1101,台泥,11938157,5094,615005211,51.50,51.80,51.30,51.60,+,0.10,51.50,325,51.60,114,12.34,
2,1101B,台泥乙特,0,0,0,--,--,--,--,,0.00,53.40,2,53.90,15,0.00,
3,1102,亞泥,8267759,3635,430280018,52.50,52.60,51.80,51.90,-,0.60,51.90,146,52.00,2027,10.16,


In [4]:
# step 4. basic analysis

df.shape
df.describe()
#df.to_csv('c:\Python\datestr.csv')

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,
count,983,983,983,983,983,983,983,983,983,983,983,983,983,983,983,983,983
unique,983,983,981,722,982,737,741,731,750,4,122,745,204,720,177,721,1
top,1101,台泥,0,14,0,11.90,68.50,62.80,20.75,+,0.00,14.75,1,--,1,0.00,
freq,1,1,2,6,2,6,4,5,5,695,97,5,127,30,126,178,983


In [5]:
df[:3]

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,
1,1101,台泥,11938157,5094,615005211,51.50,51.80,51.30,51.60,+,0.10,51.50,325,51.60,114,12.34,
2,1101B,台泥乙特,0,0,0,--,--,--,--,,0.00,53.40,2,53.90,15,0.00,
3,1102,亞泥,8267759,3635,430280018,52.50,52.60,51.80,51.90,-,0.60,51.90,146,52.00,2027,10.16,


In [7]:
# step 5. append data to Azure SQL Server (Error:缺少套件)

import pyodbc
from sqlalchemy import create_engine
import urllib

params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);Database=tempdb;UID=sadmin;PWD=Pa$$w0rd@2021')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)

#將資料回寫後端資料庫(會重複寫入，因為它沒有判斷能力)
df.to_sql(name = 'tblTWSE', con = engine, if_exists = 'append', index = False, schema = "dbo")

ModuleNotFoundError: No module named 'sqlalchemy'

In [8]:
!pip install sqlalchemy

     ---------------------------------------- 1.9/1.9 MB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 192.1/192.1 kB ? eta 0:00:00


In [9]:
# step 5. append data to Azure SQL Server (Error:名稱裡有空白)

import pyodbc
from sqlalchemy import create_engine
import urllib

params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);Database=tempdb;UID=sadmin;PWD=Pa$$w0rd@2021')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)

#將資料回寫後端資料庫(會重複寫入，因為它沒有判斷能力)
df.to_sql(name = 'tblTWSE', con = engine, if_exists = 'append', index = False, schema = "dbo")

ArgumentError: Column must be constructed with a non-blank name or assign a non-blank .name before adding to a Table.

In [10]:
df.columns

Index(['證券代號', '證券名稱', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價',
       '漲跌(+/-)', '漲跌價差', '最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量', '本益比', ''],
      dtype='object', name=0)

In [11]:
# step 5. append data to Azure SQL Server

import pyodbc
from sqlalchemy import create_engine
import urllib

params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);Database=tempdb;UID=sadmin;PWD=Pa$$w0rd@2021')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)

#將資料回寫後端資料庫(會重複寫入，因為它沒有判斷能力)
df.iloc[:, :-1].to_sql(name = 'tblTWSE', con = engine, if_exists = 'append', index = False, schema = "dbo")

66

In [12]:
# step 6. python查詢SQL Server (Error:證券名稱未正確)

import pyodbc 
import pandas as pd

conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);Database=tempdb;UID=sadmin;PWD=Pa$$w0rd@2021")      
stmt = """
SELECT * from tblTWSE
"""
dfazure = pd.read_sql(stmt, conn)
dfazure

C:\Users\sadmin\AppData\Local\Temp\2\ipykernel_8072\1701666716.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfazure = pd.read_sql(stmt, conn)


,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,1101,??,11938157,5094,615005211,51.50,51.80,51.30,51.60,+,0.10,51.50,325,51.60,114,12.34
1,1101B,????,0,0,0,--,--,--,--,,0.00,53.40,2,53.90,15,0.00
2,1102,??,8267759,3635,430280018,52.50,52.60,51.80,51.90,-,0.60,51.90,146,52.00,2027,10.16
3,1103,??,635035,446,14180092,22.30,22.45,22.25,22.35,+,0.10,22.35,4,22.40,11,7.79
4,1104,??,661056,421,15094002,23.05,23.05,22.75,23.00,+,0.05,22.95,110,23.00,15,10.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,9944,??,824355,366,18495576,21.85,22.80,21.85,22.55,+,0.70,22.50,2,22.55,1,30.47
979,9945,???,5598600,3203,361218660,63.80,64.90,63.80,64.80,+,1.50,64.70,42,64.80,83,7.95
980,9946,????,50223,57,740196,14.70,14.80,14.70,14.75,+,0.05,14.75,6,14.80,10,92.19
981,9955,??,256031,182,4733216,18.10,18.70,18.05,18.60,+,0.60,18.60,8,18.65,3,0.00


In [13]:
# back to step 5. append data to Azure SQL Server

import pyodbc
from sqlalchemy import create_engine
import urllib

params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);Database=tempdb;UID=sadmin;PWD=Pa$$w0rd@2021')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = create_engine(conn_str)

#將資料回寫後端資料庫(會重複寫入，因為它沒有判斷能力)
df.iloc[:, :-1].to_sql(name = 'tblTWSE', con = engine, if_exists = 'append', index = False, schema = "dbo")

66

In [14]:
# start again step 6. python查詢SQL Server（待解決“證券名稱”問題）

import pyodbc 
import pandas as pd

conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);Database=tempdb;UID=sadmin;PWD=Pa$$w0rd@2021")      
stmt = """
SELECT * from tblTWSE
"""
dfazure = pd.read_sql(stmt, conn)
dfazure

C:\Users\sadmin\AppData\Local\Temp\2\ipykernel_8072\1303063906.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfazure = pd.read_sql(stmt, conn)


,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,1101,??,11938157,5094,615005211,51.50,51.80,51.30,51.60,+,0.10,51.50,325,51.60,114,12.34
1,1101B,????,0,0,0,--,--,--,--,,0.00,53.40,2,53.90,15,0.00
2,1102,??,8267759,3635,430280018,52.50,52.60,51.80,51.90,-,0.60,51.90,146,52.00,2027,10.16
3,1103,??,635035,446,14180092,22.30,22.45,22.25,22.35,+,0.10,22.35,4,22.40,11,7.79
4,1104,??,661056,421,15094002,23.05,23.05,22.75,23.00,+,0.05,22.95,110,23.00,15,10.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,9944,??,824355,366,18495576,21.85,22.80,21.85,22.55,+,0.70,22.50,2,22.55,1,30.47
979,9945,???,5598600,3203,361218660,63.80,64.90,63.80,64.80,+,1.50,64.70,42,64.80,83,7.95
980,9946,????,50223,57,740196,14.70,14.80,14.70,14.75,+,0.05,14.75,6,14.80,10,92.19
981,9955,??,256031,182,4733216,18.10,18.70,18.05,18.60,+,0.60,18.60,8,18.65,3,0.00
